In [1]:
pip install aesara

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
# import snowflake.connector
# from snowflake.connector.pandas_tools import write_pandas
import getpass
import datetime as dt
import seaborn as sns
import boto3
from sagemaker import get_execution_role
import sklearn.metrics as metrics

import arviz as az
import theano.tensor as tt
import theano
import aesara.tensor as at

from pymc import *
import pymc as pm

import random
import os

In [3]:
print(az.__version__)
print(theano.__version__)
print(pm.__version__)

0.12.1
1.1.2
4.0.0b6


In [4]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

# Getting Data

In [5]:
role = get_execution_role()
bucket = 'decked-mmm'
data_key = 'df8.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df = pd.read_csv(data_location)

In [6]:
data_key = 'coef.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

coef = pd.read_csv(data_location)

In [7]:
coef

,Unnamed: 0,features,coef
0,0,PODCAST,-88470.877779
1,1,SERP,-67240.759883
2,2,MAJOR_CHANGES_10,-44201.875738
3,3,BING,-42820.207502
4,4,MONTHNAME_Jan,-42189.914459
5,5,MONTHNAME_Sep,-38309.155641
6,6,LINKTREE_SESSION,-31620.321007
7,7,MONTHNAME_Jun,-26237.209054
8,8,MONTHNAME_Aug,-23738.867988
9,9,STEELHOUSE_RETARGETING,-23005.184082


In [8]:
df['DATE_AT'].min()

'2020-03-01'

In [9]:
df['DATE_AT'].max()

'2022-10-30'

In [10]:
df = df.loc[df['DATE_AT']>'2020-08-31']

In [11]:
df[['STEELHOUSE_PROSPECTING', 'SALES']].describe()

,STEELHOUSE_PROSPECTING,SALES
count,113.000000,113.000000
mean,0.350758,601803.379381
std,0.214972,112894.600495
min,0.000000,90842.090000
25%,0.243299,527291.990000
50%,0.337936,599685.250000
75%,0.503377,688501.810000
max,1.000000,896055.900000


# Modeling

In [12]:
p = {'STEELHOUSE_PROSPECTING':0.5
     , 'STEELHOUSE_RETARGETING':0
     , 'ADWORDS_YT':1.5
     , 'FB_PROSPECTING':1.5
     , 'OPENS_EMAIL':1
     , 'PODCAST':1.5
     , 'YOUTUBE_SHOWS': 1.5
     , 'IMPRESSIONS_ONLINE_PUBS': 1.5
     , 'SEARCH_INDEX':0
     , 'SERP':1}

d = {'STEELHOUSE_PROSPECTING':0.05
     , 'STEELHOUSE_RETARGETING':0.3
     , 'ADWORDS_YT':0.3
     , 'FB_PROSPECTING':0.05
     , 'OPENS_EMAIL':0.5
     , 'PODCAST':0.3
     , 'YOUTUBE_SHOWS':0.3
     , 'IMPRESSIONS_ONLINE_PUBS':0.3
     , 'SEARCH_INDEX':0.5
     , 'SERP':0.3}

In [15]:
# defining function

def logistic_function(x_t, mu):
    return (1 - np.exp(-mu * x_t)) / (1 + np.exp(-mu * x_t))

def geometric_adstock_tt(x_t, alpha=0, L=12, normalize=True):
    w = at.as_tensor_variable([at.power(alpha, i) for i in range(L)])
    xx = at.stack([at.concatenate([at.zeros(i), x_t[:x_t.shape[0] - i]]) for i in range(L)])
    
    if not normalize:
        y = at.dot(w, xx)
    else:
        y = at.dot(w / at.sum(w), xx)
    return y

def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)    
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [16]:
# transformations

# delay
delay_channels = ['STEELHOUSE_PROSPECTING',
                  'ADWORDS_YT',
                  'OPENS_EMAIL',
                  'PODCAST',
                  'YOUTUBE_SHOWS',
                  'IMPRESSIONS_ONLINE_PUBS',
                  'SERP',
                  'STEELHOUSE_RETARGETING',
                  'FB_RETARGETING',
                  'SEARCH_INDEX']
delay_priors = [0.5, 1.5, 1, 1.5, 1.5, 1.5, 1, 0, 0, 0]
norm_delay_priors = normalize(delay_priors, 1, 5)
#norm_delay_priors = [0.5, 1.5, 1, 1.5, 1.5, 1.5, 1, 0.01, 0.01, 0.01]
delay_dict = {channel: prior for channel, prior in zip(delay_channels, norm_delay_priors)}
print(delay_dict)

# saturation
non_lin_channels = ['ADWORDS_LUMP',
                    'ADWORDS_SEARCH', 
                    'PERFMAX', 
                    'BING', 
                    'FB_PROSPECTING',
                    'IMPRESSIONS_BRANDED_SEARCH',
                    'IMPRESSIONS_NONBRANDED_SEARCH',]

# control
control_vars = ['NEW_VEHICLE_SALES', 'USED_VEHICLE_SALES', 'LINKTREE_SESSION',
       'PRODUCT_LAUNCH', 'MAJOR_CHANGES', 'AOV',
       'NEW_VEHICLE_SALES_8', 'NEW_VEHICLE_SALES_14',
       'USED_VEHICLE_SALES_12', 'USED_VEHICLE_SALES_16', 'PRODUCT_LAUNCH_8',
       'MAJOR_CHANGES_6', 'MAJOR_CHANGES_10', 'MAJOR_CHANGES_12', 'ISHOLIDAY']
index_vars = ['MONTHNAME_Apr', 'MONTHNAME_Aug',
       'MONTHNAME_Dec', 'MONTHNAME_Feb', 'MONTHNAME_Jul',
       'MONTHNAME_Jun', 'MONTHNAME_Mar', 'MONTHNAME_May', 'MONTHNAME_Nov',
       'MONTHNAME_Oct', 'MONTHNAME_Sep']
                    
#outcome
outcome = 'SALES'

{'STEELHOUSE_PROSPECTING': 2.333333333333333, 'ADWORDS_YT': 5.0, 'OPENS_EMAIL': 3.6666666666666665, 'PODCAST': 5.0, 'YOUTUBE_SHOWS': 5.0, 'IMPRESSIONS_ONLINE_PUBS': 5.0, 'SERP': 3.6666666666666665, 'STEELHOUSE_RETARGETING': 1.0, 'FB_RETARGETING': 1.0, 'SEARCH_INDEX': 1.0}


In [26]:
# modeling
df_train = df.drop(['Unnamed: 0', 'DATE_AT'], axis=1)

with Model() as model:
    response_mean = []
    x_ = pm.MutableData('features', df_train) # a data container, can be changed
    t = np.transpose(x_.get_value())
    
    # intercept
    y = Normal('y', mu=0, sigma=6000)
    response_mean.append(y)
    
    # channels that can have DECAY and SATURATION effects
    for channel_name in delay_channels:
        i = df_train.columns.get_loc(channel_name)
        xx = t[i].astype(float)
        
        print(f'Adding Delayed Channels: {channel_name}')
        c = coef.loc[coef['features']==channel_name, 'coef'].values[0]
        s = abs(c*0.015)
        if c <= 0:
            channel_b = HalfNormal(f'beta_{channel_name}', sigma=s)
        else:
            channel_b = Normal(f'beta_{channel_name}', mu=c, sigma=s)
        
        alpha = pm.Beta(f'alpha_{channel_name}', alpha=3, beta=3)
        channel_mu = Gamma(f'mu_{channel_name}', alpha=3, beta=1)
        response_mean.append(logistic_function(
            geometric_adstock_tt(xx, alpha), channel_mu) * channel_b)
    
    # channels that have SATURATION effects only
    for channel_name in non_lin_channels:
        i = df_train.columns.get_loc(channel_name)
        xx = t[i].astype(float)
        
        print(f'Adding Non-Linear Logistic Channel: {channel_name}')
        c = coef.loc[coef['features']==channel_name, 'coef'].values[0]
        s = abs(c*0.015)
        if c <= 0:
            channel_b = HalfNormal(f'beta_{channel_name}', sigma=s)
        else:
            channel_b = Normal(f'beta_{channel_name}', mu=c, sigma=s)
        
        # logistic reach curve
        channel_mu = Gamma(f'mu_{channel_name}', alpha=3, beta=1)
        response_mean.append(logistic_function(xx, channel_mu) * channel_b)
        
    # continuous external features
    for channel_name in control_vars:
        i = df_train.columns.get_loc(channel_name)
        xx = t[i].astype(float)

        print(f'Adding control: {channel_name}')
        c = coef.loc[coef['features']==channel_name, 'coef'].values[0]
        s = abs(c*0.015)
        if c <= 0:
            control_beta = HalfNormal(f'beta_{channel_name}', sigma=s)
        else:
            control_beta = Normal(f'beta_{channel_name}', mu=c, sigma=s)
            
        channel_contrib = control_beta * xx
        response_mean.append(channel_contrib)
        
    # categorical control variables
    for var_name in index_vars:
        i = df_train.columns.get_loc(var_name)
        shape = len(np.unique(t[i]))
        x = t[i].astype('int')
        
        print(f'Adding Index Variable: {var_name}')
        
        ind_beta = Normal(f'beta_{var_name}', sigma=6000, shape=shape)
        channel_contrib = ind_beta[x]
        response_mean.append(channel_contrib)
        
    # noise
    sigma = Exponential('sigma', 10)

    
    # define likelihood
    likelihood = Normal(outcome, mu=sum(response_mean), sigma=sigma, observed=df[outcome].values)
    
    trace = pm.sample(3000,tune=500, init="auto")

Adding Delayed Channels: STEELHOUSE_PROSPECTING
Adding Delayed Channels: ADWORDS_YT
Adding Delayed Channels: OPENS_EMAIL
Adding Delayed Channels: PODCAST
Adding Delayed Channels: YOUTUBE_SHOWS
Adding Delayed Channels: IMPRESSIONS_ONLINE_PUBS
Adding Delayed Channels: SERP
Adding Delayed Channels: STEELHOUSE_RETARGETING
Adding Delayed Channels: FB_RETARGETING
Adding Delayed Channels: SEARCH_INDEX
Adding Non-Linear Logistic Channel: ADWORDS_LUMP
Adding Non-Linear Logistic Channel: ADWORDS_SEARCH
Adding Non-Linear Logistic Channel: PERFMAX
Adding Non-Linear Logistic Channel: BING
Adding Non-Linear Logistic Channel: FB_PROSPECTING
Adding Non-Linear Logistic Channel: IMPRESSIONS_BRANDED_SEARCH
Adding Non-Linear Logistic Channel: IMPRESSIONS_NONBRANDED_SEARCH
Adding control: NEW_VEHICLE_SALES
Adding control: USED_VEHICLE_SALES
Adding control: LINKTREE_SESSION
Adding control: PRODUCT_LAUNCH
Adding control: MAJOR_CHANGES
Adding control: AOV
Adding control: NEW_VEHICLE_SALES_8
Adding control: NE

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pymc/aesaraf.py:1005: UserWarning: The parameter 'updates' of aesara.function() expects an OrderedDict, got <class 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (collections.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.
  aesara_function = aesara.function(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [y, beta_STEELHOUSE_PROSPECTING, alpha_STEELHOUSE_PROSPECTING, mu_STEELHOUSE_PROSPECTING, beta_ADWORDS_YT, alpha_ADWORDS_YT, mu_ADWORDS_YT, beta_OPENS_EMAIL, alpha_OPENS_EMAIL, mu_OPENS_EMAIL, beta_PODCAST, alpha_PODCAST, mu_PODCAST, beta_YOUTUBE_SHOWS, alpha_YOUTUBE_SHOWS, mu_YOUTUBE_SHOWS, beta_IMPRESSIONS_ONLINE_

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered in true_divide
  np.divide(1, self._stds, out=self._inv_stds)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pymc/step_methods/hmc/quadpotential.py:237: RuntimeWarning: invalid value encountered in multiply
  return np.multiply(self._var, x, out=out)
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pymc/step_methods/hmc/quadpotential.py:258: RuntimeWarning: divide by zero encountered i

RuntimeError: Chain 2 failed.

In [ ]:
alphas = mus = [f'alpha_{n}' for n in delay_channels]
mus = [f'mu_{n}' for n in delay_channels + non_lin_channels]
#betas = [f'beta_{n}' for n in delay_channels + non_lin_channels + control_vars + index_vars]
betas = [f'beta_{n}' for n in delay_channels + non_lin_channels + control_vars]

In [ ]:
slope = az.summary(trace, var_names=betas, kind="stats")
slope.sort_values('mean', ascending=False)

In [ ]:
_ = az.plot_trace(trace, var_names=betas, legend=True)

In [ ]:
_ = az.plot_trace(trace, var_names=alphas)

In [ ]:
_ = az.plot_trace(trace, var_names=mus)

In [ ]:
with model:
    y_pred = sample_posterior_predictive(trace)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
y1 = df.reset_index()['SALES']
y2 = y_pred['SALES'].mean(axis=0)
y3 = np.median(y_pred['SALES'], axis=0)
plt.plot(y1, label = 'Actual')
plt.plot(y2, label = 'Predicted_mean')
plt.plot(y3, label = 'Predicted_median')
plt.legend()
plt.show()

In [ ]:
resid = y1 - y2
resid.hist(bins=100)
plt.show()

In [ ]:
#regression results
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred)
    mape=np.mean(np.abs(y_true - y_pred) / np.abs(y_true)) * 100
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MAPE: ', round(mape,4), ' %')
    
regression_results(y1, y2)

In [ ]:
#slope
slope = slope.sort_values('mean', ascending=False)
slope.to_csv('slope_v3_2yrs.csv')

In [ ]:
#params
coeffs = {}
for item in (alphas + betas + mus):
    coeffs[item] =  trace[item].mean()
params = pd.DataFrame(coeffs.items())
params.to_csv('params_v3_2yrs.csv')

param_sum = az.summary(trace)
param_sum.to_csv('param_sum_v3_2yrs.csv')

# ROAS Calc

In [ ]:
mod_channel = 'ADWORDS_LUMP'
df_mod = df_train.copy(deep=True)
df_mod.iloc[12:-12, df_mod.columns.get_loc(mod_channel)] = 0

In [125]:
#x_.set_value(df_mod)

In [ ]:
with model:
    pm.set_data({'features':df_mod})
    y_pred_mod = pm.sample_posterior_predictive(trace)

In [ ]:
df_og = df_train.copy(deep=True)
with model:
    pm.set_data({'features':df_og})
    y_pred_og = sample_posterior_predictive(trace)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
y1_ = y_pred['SALES'].mean(axis=0)
y2_ = y_pred_mod['SALES'].mean(axis=0)
plt.plot(y1_, label = 'og')
plt.plot(y2_, label = 'mod')
plt.legend()
plt.show()

In [ ]:
y1_.sum() - y2_.sum()

In [ ]:
y1_ - y2_

In [ ]:
y3_ = y_pred_og['SALES'].mean(axis=0)
y1_.sum() - y3_.sum()

In [ ]:
y1_ - y3_

In [117]:
trace_p = pm.save_trace(trace, overwrite=True, directory='/home/ec2-user/SageMaker/trace')

In [83]:
with model:
    trace2 = pm.load_trace(trace_p)